In [1]:
from __future__ import print_function
import os
import re
import scipy.misc
import warnings
import face_recognition.api as face_recognition
import sys
import cv2

In [2]:
def scan_known_people(known_people_folder):
    known_names = []
    known_face_encodings = []

    basename = known_people_folder 
    img = face_recognition.load_image_file(known_people_folder)
    encodings = face_recognition.face_encodings(img)
    if len(encodings) == 1:
        known_names.append(basename)
        known_face_encodings.append(encodings[0])   
    return known_names, known_face_encodings


In [3]:
def test_image(image_to_check, known_names, known_face_encodings):
    unknown_image = face_recognition.load_image_file(image_to_check)

    # Scale down image if it's giant so things run a little faster
    if unknown_image.shape[1] > 1600:
        scale_factor = 1600.0 / unknown_image.shape[1]
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            unknown_image = scipy.misc.imresize(unknown_image, scale_factor)

    unknown_encodings = face_recognition.face_encodings(unknown_image)
    if len(unknown_encodings)==1:
        for unknown_encoding in unknown_encodings:
            result = face_recognition.compare_faces(known_face_encodings, unknown_encoding)
            distance = face_recognition.face_distance(known_face_encodings, unknown_encoding)
            # print(distance[0])
            print("Face Matched!") if True in result else print("Face not matched")

        return distance[0],result[0]
    else:
        return "0","Many Faces or No Faces"


In [4]:
def image_files_in_folder(folder):
    return [os.path.join(folder, f) for f in os.listdir(folder) if re.match(r'.*\.(jpg|jpeg|png)', f, flags=re.I)]

In [5]:
def main(known_people_folder, image_to_check):
    known_names, known_face_encodings = scan_known_people(known_people_folder)
    distance,result=test_image(image_to_check, known_names, known_face_encodings)
    return distance,result


In [7]:
def get_coordinates(img):
    #img = face_recognition.load_image_file(image_path)
    locations = face_recognition.face_locations(img)
    location_json=[]
    for coordinates in locations:
        json_dict={
            'ymin':coordinates[0],
            'xmin':coordinates[1],
            'ymax':coordinates[2],
            'xmax':coordinates[3]}

        location_json.append(json_dict)
    return locations,location_json

In [19]:
cam = cv2.VideoCapture(0)

cv2.namedWindow("test")

img_counter = 0

while True:
    ret, frame = cam.read()
    if not ret:
        print("failed to grab frame")
        break
    cv2.imshow("test", frame)

    k = cv2.waitKey(1)
    if k%256 == 27:
        # ESC pressed
        print("Escape hit, closing...")
        break
    elif k%256 == 32:
        # SPACE pressed
        img_name = "opencv_frame_{}.jpg".format(img_counter)
        cv2.imwrite(img_name, frame)
        print("{} written!".format(img_name))
        img_counter += 1  
        
cam.release()

cv2.destroyAllWindows()

opencv_frame_0.jpg written!
opencv_frame_1.jpg written!
Escape hit, closing...


In [20]:
import numpy as np
id_path = "opencv_frame_0.jpg"
selfie_path = "opencv_frame_1.jpg"
distance, result = main(id_path, selfie_path)
print(result)
img_id = cv2.imread(id_path)
img_selfie = cv2.imread(selfie_path)
#concatanate image Horizontally
img_concate_Hori=np.concatenate((img_id,img_selfie),axis=1)
res,_=get_coordinates(img_concate_Hori)
if result == True:
    label = "Verified. Faces Matched!"
else:
    label = "Sorry! Cannot verify. Faces don't matched."
for coordinates in res:
    ymin, xmin, ymax, xmax = coordinates
    cv2.rectangle(img_concate_Hori, (xmin, ymin), (xmax, ymax), (255,255,0), 2)
cv2.putText(img_concate_Hori, label, (250,450), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)
cv2.imshow('test', img_concate_Hori) 
k = cv2.waitKey(0) & 0xFF
    
# wait for ESC key to exit 
if k == 27:  
    cv2.destroyAllWindows() 
        
# wait for 's' key to save and exit 
elif k == ord('s'):  
    cv2.imwrite('result.jpg',img) 
    cv2.destroyAllWindows() 

Face not matched
False
